In [1]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as tf
from PIL import Image
import torch.autograd as autograd

The images files have the form "ID_2m_0P_xV_yH_z.jpg" where ID is the ID of the person, 2m is fixed, 0P means head pose of 0 degrees (only head pose used in this notebook)
x is the vertical orientation, y is the horizontal orientation and z is either L for left or R for right eye (note that the right eye patch was flipped horizontally).
In training the images are grouped as follows:
For a given person and a given eye (R or L) all orientations are grouped together. One element of the data set is of the form
imgs_r,angles_r,labels,imgs_t,angles_g where imgs_r is considered the "real" image with orientation angles_r, or x_r in the paper,
imgs_t with orientation angles_g is the image of the same person with different orientation (could be the same image since we go through a double loop) and the label is the ID of the person

In [2]:

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dir_path, transform=None):
        super().__init__()
        self.transform = transform
        self.ids=50
        self.data_path = dir_path
        self.file_names = [f for f in os.listdir(self.data_path)
                      if f.endswith('.jpg')]
        self.file_dict = dict()
        for f_name in self.file_names:
            fields = f_name.split('.')[0].split('_')
            identity = fields[0]
            head_pose = fields[2]
            side = fields[-1]
            key = '_'.join([identity, head_pose, side])
            if key not in self.file_dict.keys():
                self.file_dict[key] = []
                self.file_dict[key].append(f_name)
            else:
                self.file_dict[key].append(f_name)
        self.train_images = []
        self.train_angles_r = []
        self.train_labels = []
        self.train_images_t = []
        self.train_angles_g = []

        self.test_images = []
        self.test_angles_r = []
        self.test_labels = []
        self.test_images_t = []
        self.test_angles_g = []
        self.preprocess()
    def preprocess(self):

        for key in self.file_dict.keys():

            if len(self.file_dict[key]) == 1:
                continue

            idx = int(key.split('_')[0])
            flip = 1
            if key.split('_')[-1] == 'R':
                flip = -1

            for f_r in self.file_dict[key]:

                file_path = os.path.join(self.data_path, f_r)

                h_angle_r = flip * float(
                    f_r.split('_')[-2].split('H')[0]) / 15.0
                    
                v_angle_r = float(
                    f_r.split('_')[-3].split('V')[0]) / 10.0
                    

                for f_g in self.file_dict[key]:

                    file_path_t = os.path.join(self.data_path, f_g)

                    h_angle_g = flip * float(
                        f_g.split('_')[-2].split('H')[0]) / 15.0
                        
                    v_angle_g = float(
                        f_g.split('_')[-3].split('V')[0]) / 10.0
                        

                    if idx <= self.ids:
                        self.train_images.append(file_path)
                        self.train_angles_r.append([h_angle_r, v_angle_r])
                        self.train_labels.append(idx - 1)
                        self.train_images_t.append(file_path_t)
                        self.train_angles_g.append([h_angle_g, v_angle_g])
                    else:
                        self.test_images.append(file_path)
                        self.test_angles_r.append([h_angle_r, v_angle_r])
                        self.test_labels.append(idx - 1)
                        self.test_images_t.append(file_path_t)
                        self.test_angles_g.append([h_angle_g, v_angle_g])

    def __getitem__(self, index):
        return (
            self.transform(Image.open(self.train_images[index])),
                torch.tensor(self.train_angles_r[index]),
                self.train_labels[index],
            self.transform(Image.open(self.train_images_t[index])),
                torch.tensor(self.train_angles_g[index]))
        
    def __len__(self):
        return len(self.train_images)
    

In [3]:
transform=tf.Compose([tf.ToTensor(),tf.Resize((64,64),antialias=True)])
# dataset=MyDataset(dir_path='/home/user/Downloads/dataset/0P',transform=transform)
dataset=MyDataset(dir_path='/home/user/Downloads/dataset/0P',transform=transform)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [5]:
imgs_r,angles_r,labels,imgs_g,angles_g=next(iter(train_loader))
print(imgs_r.shape,angles_r.shape,labels.shape,imgs_g.shape,angles_g.shape)

torch.Size([32, 3, 64, 64]) torch.Size([32, 2]) torch.Size([32]) torch.Size([32, 3, 64, 64]) torch.Size([32, 2])


In [6]:
device='cuda' if torch.cuda.is_available() else 'cpu'
#device='cpu'

In [7]:
from networks import Discriminator,Generator2
# if os.path.isfile('discriminator.pth'):
#     discriminator=torch.load('discriminator.pth')
#     print('loaded discriminator')
# else:
#     discriminator=Discriminator()
#     print('created discriminator')
# if os.path.isfile('generator.pth'):
#     generator=torch.load('generator.pth')
#     print('loaded generator')
# else:
#     generator=Generator2()
#     print('created generator')

discriminator=Discriminator()
generator=Generator2()
generator=generator.to(device)
discriminator=discriminator.to(device)
LR = 5e-5
beta1=0.5
beta2=0.999
optimizer_g = torch.optim.Adam(generator.parameters(), LR,betas=(beta1, beta2))
optimizer_d = torch.optim.Adam(discriminator.parameters(), LR,betas=(beta1, beta2))

In [8]:
from loss_network import LossNetwork
loss_network=LossNetwork()
loss_network=loss_network.to(device)


/home/user/anaconda3/envs/pytorch2/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/anaconda3/envs/pytorch2/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
from loss import content_style_loss,adv_loss_d,adv_loss_g,gaze_loss_d,gaze_loss_g,reconstruction_loss

In [10]:
def generator_step(generator,discriminator,loss_network,imgs_r,imgs_t,angles_r,angles_g):
    optimizer_g.zero_grad()
    generator.train()
    discriminator.eval()
    x_g=generator(imgs_r,angles_g)
    x_recon=generator(x_g,angles_r)
    loss_adv=adv_loss_g(discriminator,imgs_r,x_g)
    loss2=content_style_loss(loss_network,x_g,imgs_t)
    loss_p=loss2[0]+loss2[1]
    loss_gg=gaze_loss_g(discriminator,x_g,angles_g)
    loss_recon=reconstruction_loss(generator,imgs_r,x_recon)
    loss=loss_adv+100*loss_p+5*loss_gg+50*loss_recon
    loss.backward()
    optimizer_g.step()
    return loss.item()

In [11]:
def discriminator_step(generator,discriminator,imgs_r,imgs_t,angles_r,angles_g):
    optimizer_d.zero_grad()
    generator.eval()
    discriminator.train()
    x_g=generator(imgs_r,angles_g)
    loss1=adv_loss_d(discriminator,imgs_r,x_g)
    loss2=gaze_loss_d(discriminator,imgs_r,angles_r)
    loss=loss1+5*loss2
    loss.backward()
    optimizer_d.step()
    return loss.item()

In [12]:
from PIL import Image
import numpy as np
def recover_image(img):
    img=img.cpu().numpy().transpose(0, 2, 3, 1)*255
    return img.astype(np.uint8)
def save_images(imgs, filename):
    height=recover_image(imgs[0])[0].shape[0]
    width=recover_image(imgs[0])[0].shape[1]
    total_width=width*len(imgs)
    
    new_im = Image.new('RGB', (total_width+len(imgs), height))
    for i,img in enumerate(imgs):
        result = Image.fromarray(recover_image(img)[0])
        new_im.paste(result, (i*width+i,0))
    new_im.save(filename)

In [13]:
!mkdir -p debug

/bin/bash: /home/user/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [14]:

epochs=300
for epoch in range(epochs):
    count=0
  
    for imgs_r, angles_r, labels, imgs_t, angles_g in train_loader:
        count+=1
        imgs_r=imgs_r.to(device)
        imgs_t=imgs_t.to(device)
        angles_r=angles_r.to(device)
        angles_g=angles_g.to(device)
        l_d=discriminator_step(generator,discriminator,imgs_r,imgs_t,angles_r,angles_g)
        if count%5==0:
            l_g=generator_step(generator,discriminator,loss_network,imgs_r,imgs_t,angles_r,angles_g)
        if count%1000==0:
            imgs=[imgs_r]
            for h in [-15,-10,-5,0,5,10,15]:
                    a=torch.tile(torch.tensor([h/15.,0.]),[32,1])
                    a=a.to(device)
                    y=generator(imgs_r,a)
                    imgs.append(y.detach())
            save_images(imgs, "./debug/{}_{}.png".format(epoch,count))
    print(l_d,l_g)
    if epoch%20==0:
        torch.save(generator, './generator.pth')
        torch.save(discriminator, './discriminator.pth')
     
    

-0.31358757615089417 2.9682517051696777
-0.2342233806848526 2.3327081203460693
-0.2994817793369293 1.940845012664795
-0.3117067217826843 1.8902649879455566
-0.35777783393859863 1.546557068824768
-0.37074172496795654 1.5946787595748901
-0.3110412061214447 1.4235047101974487
-0.3506832420825958 1.4569635391235352
-0.27027928829193115 1.416990876197815
-0.3218216896057129 1.3143155574798584
-0.24718238413333893 1.4948667287826538
-0.2606305778026581 1.148314118385315
-0.2549508810043335 1.1774322986602783
-0.25507697463035583 1.0714054107666016
-0.19905276596546173 1.139740228652954
-0.3423738479614258 1.3190399408340454
-0.39515891671180725 1.027995228767395
-0.29178595542907715 1.0882527828216553
-0.26527005434036255 1.256840467453003
-0.18343782424926758 1.3145817518234253
-0.27738529443740845 0.972407877445221
-0.3530397415161133 1.5277353525161743
-0.21616946160793304 1.0154327154159546
-0.294808030128479 0.9910871386528015
-0.1701388657093048 1.01249361038208
-0.21900419890880585 0.

In [15]:
a=torch.tile(torch.tensor([0.,0.]),[32,1])
print(a.size(),angles_r.size())
y=generator(imgs_r.to(device),angles_r.to(device))

torch.Size([32, 2]) torch.Size([4, 2])
